In [ ]:
import sys
!{sys.executable} -m pip install SPARQLWrapper


In [22]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [32]:
df = pd.read_csv("dbpedia.tsv", sep='\t', names=['id','movie','uri'], header=None)
df.head(10)

,id,movie,uri
0,781,Stealing Beauty (1996),http://dbpedia.org/resource/Stealing_Beauty
1,1799,Suicide Kings (1997),http://dbpedia.org/resource/Suicide_Kings
2,521,Romeo Is Bleeding (1993),http://dbpedia.org/resource/Romeo_Is_Bleeding
3,3596,Screwed (2000),http://dbpedia.org/resource/Screwed_(2000_film)
4,3682,Magnum Force (1973),http://dbpedia.org/resource/Magnum_Force
5,2635,"Mummy's Curse, The (1944)",http://dbpedia.org/resource/The_Mummy's_Curse
6,1590,Event Horizon (1997),http://dbpedia.org/resource/Event_Horizon_(film)
7,116,Anne Frank Remembered (1995),http://dbpedia.org/resource/Anne_Frank_Remembered
8,3719,Love's Labour's Lost (2000),http://dbpedia.org/resource/Love's_Labour's_Lo...
9,3811,Breaker Morant (1980),http://dbpedia.org/resource/Breaker_Morant_(film)


In [24]:

#Query for Starrings
#print("Starrings")

#uri="http://dbpedia.org/resource/Event_Horizon_(film)"
uri="http://dbpedia.org/resource/Romeo_Is_Bleeding"

dict_info = {}
def query_sparql(URI):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
        PREFIX dbpediaO: <http://dbpedia.org/ontology/>
        SELECT ?list ?name
        WHERE {
         VALUES ?s {<%s>}
         ?s dbpediaO:starring ?list .
         ?list foaf:name ?name . }
    """ % URI)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    dict_info['starring']=[]
    for result in results["results"]["bindings"]:
        dict_info['starring'].append(result["name"]["value"])
        #print(result["name"]["value"])
        
    #Query for director
    #print("\nDirector")
    sparql.setQuery("""
        PREFIX dbpediaO: <http://dbpedia.org/ontology/>
        SELECT ?list ?name
        WHERE {
         VALUES ?s {<%s>}
         ?s dbpediaO:director ?list .
         ?list foaf:name ?name . }
    """ % URI)
    results = sparql.query().convert()



    for result in results["results"]["bindings"]:
        dict_info['director']=result["name"]["value"]
        #print(result["name"]["value"])
    #print(dict_info)
    return dict_info

print(query_sparql(uri))


In [28]:
db=df.head()

In [33]:
# creating a blank series 
director = pd.Series([]) 
starring = pd.Series([]) 

for i in range(len(df)):
    uri=df["uri"][i]
    dict_info = query_sparql(uri)
    director[i] = dict_info['director']
    
    #Convertir lista actores a string separados por coma
    my_string = ','.join(dict_info['starring'])
    starring[i] = my_string
    #print(uri)
    
# inserting new column with values of list made above         
df.insert(3, "director", director) 
df.insert(4, "starring", starring) 
# list output 
df.head() 

C:\Users\KaiLo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\KaiLo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


,id,movie,uri,director,starring
0,781,Stealing Beauty (1996),http://dbpedia.org/resource/Stealing_Beauty,Bernardo Bertolucci,"Sinéad Cusack,Joseph Fiennes,Rachel Weisz,Jere..."
1,1799,Suicide Kings (1997),http://dbpedia.org/resource/Suicide_Kings,Peter O'Fallon,"Henry Thomas,Denis Leary,Christopher Walken,Ja..."
2,521,Romeo Is Bleeding (1993),http://dbpedia.org/resource/Romeo_Is_Bleeding,Peter Medak,"Roy Scheider,Annabella Sciorra,Lena Olin,Gary ..."
3,3596,Screwed (2000),http://dbpedia.org/resource/Screwed_(2000_film),Scott Alexander,"Sarah Silverman,Danny DeVito,Norm Macdonald,El..."
4,3682,Magnum Force (1973),http://dbpedia.org/resource/Magnum_Force,Ted Post,"Clint Eastwood,Robert Urich,David Soul,Tim Mat..."


In [35]:
csv_name = "movies1_info.csv"
df.to_csv(csv_name, index=False, sep=';')